In [2]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
import scripts.theodolite_function as theodo_f
import tqdm
import torch
from numpy import linalg
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
theodo_f = importlib.reload(theodo_f)
predict_u = importlib.reload(predict_u)

In [3]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)
theodo_g = importlib.reload(theodo_g)

## Path of main files
path = "../data/20220224/"

# Reading sensor data
# Sensor = "Robosense_32"
Sensor = "GNSS"   # GNSS or Robosense_32
# path_sensor_file = path+"ICP/icp_odom.csv"

# Sensor = "GNSS"
path_sensor_file = path+"gps_data/"+"gps3.txt"
path_sensor_file_synch_time = path+"gps_data/delay_synchronization_GNSS_3.txt"  # If applied to GNSS
Gps_reference_chosen = 3    # 1: front, 2: back, 3: middle   # Only for GNSS

## Parameters for pre-processing pipeline
file = [
        "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag"
]
parameters = [
    [1,2,1,1,3,2],  # 1. Apply filtering or not (Module 1), 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1), 5. Parameter tau_s (Module 2), 6 Parameter tau_l (Module 4).
]
output = [
        path
]

## Read sensor data which we want a ground truth
sensor_data = []
if Sensor == "GNSS":
    GNSS_raw_data = theodo_u.read_prediction_data_Linear_csv_file(path_sensor_file)
    time_delay = float(theodo_u.read_time_delay(path_sensor_file_synch_time))

    for i in GNSS_raw_data:
        raw_data = np.array([i[0] + time_delay, i[1], i[2], i[3], 0, 0, 0, 1])
        sensor_data.append(raw_data)
    sensor_data = np.array(sensor_data)
if Sensor == "Robosense_32":
    sensor_data = theodo_u.read_icp_odom_file(path_sensor_file)
    sensor_data = np.array(sensor_data)

## Read Extrinsic results for RTS
## Estimated extrinsic calibration uncertainty
number_test = 200
path_file_GCP = "total_stations/GCP.txt"
Tf_1, Tf_2, Tf_3, std1, std2, std3 = theodo_g.function_noise_resection(path+path_file_GCP, number_test)
## Number of rigid transforms to apply the uncertainty
num_samples = 1000
T_1_corrected = theodo_g.correct_tf(Tf_1, std1, num_samples)
T_2_corrected = theodo_g.correct_tf(Tf_2, std2, num_samples)
T_3_corrected = theodo_g.correct_tf(Tf_3, std3, num_samples)

for param in parameters:
    print(param)

    if param[0] == 0:
        filtering = False
    if param[0] == 1:
        filtering = True
    thresold_d = param[1]  ## tau_r [m/s]
    thresold_a = param[2]  ## tau_a [deg/s]
    thresold_e = param[3]  ## tau_e [deg/s]
    limit_time_interval = param[4]  ## tau_s [s]
    size_interval = param[5]    ## tau_l

    Mode = "STEAM"  ## Interpolation choice: 1. L -> Linear interpolation, 2. SGP -> Gaussian Process with Stheno library, 3. STEAM
    limit_search = limit_time_interval
    save = False

    save_index_1 = []
    save_index_2 = []
    save_index_3 = []

    for fname, opath in zip(file, output):
        if not filtering:
            path_out = opath + "raw_prediction/"
        else:
            path_out = opath + "filtered_prediction/"

        if filtering:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(
                fname)
            index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a * 3.1415926 / 180,
                                                   thresold_e * 3.1415926 / 180, limit_time_interval)
            index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a * 3.1415926 / 180,
                                                   thresold_e * 3.1415926 / 180, limit_time_interval)
            index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a * 3.1415926 / 180,
                                                   thresold_e * 3.1415926 / 180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1), len(t2), len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(
                fname)
            print(len(t1), len(t2), len(t3))

        list_interval, list_time = theodo_f.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = theodo_f.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []
        Index_sensor = []

        print(len(list_trajectories_split))
        nubmer_iteration = 0
        for i in list_trajectories_split:
            print(nubmer_iteration)
            nubmer_iteration = nubmer_iteration +1

            index_1 = np.array([i[0, 0], i[1, 0]])
            index_2 = np.array([i[0, 1], i[1, 1]])
            index_3 = np.array([i[0, 2], i[1, 2]])

            save_index_1.append(index_1)
            save_index_2.append(index_2)
            save_index_3.append(index_3)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if abs(end - begin) > size_interval and begin < end:

                Number = 0
                T_prediction_sensor = []
                for value_sensor_data in sensor_data:
                    if end >= value_sensor_data[0] >= begin:
                        T_prediction_sensor.append(value_sensor_data[0])
                        Index_sensor.append(Number)
                    Number = Number + 1

                # rate = 10  #Hz
                # T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))
                T_prediction_init = torch.from_numpy(np.array(T_prediction_sensor))

                # Linear interpolation
                if Mode == "L" or Mode == "All":

                    ## Put trajectories in same frame
                    tp1 = Tf_1 @ tp1
                    tp2 = Tf_2 @ tp2
                    tp3 = Tf_3 @ tp3

                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = predict_u.data_training_L(t1, t2, t3,
                                                                                      tp1, tp2, tp3,
                                                                                      index_1,
                                                                                      index_2,
                                                                                      index_3)
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = predict_u.linear_interpolation(T1, X1, Y1,
                                                                                        Z1, T2, X2,
                                                                                        Y2, Z2, T3,
                                                                                        X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i)
                        P1_L, P2_L, P3_L = predict_u.linear_prediction(i, 0, mx1, my1, mz1, mx2, my2, mz2,
                                                                          mx3, my3, mz3)
                        Prediction_1.append(P1_L)
                        Prediction_2.append(P2_L)
                        Prediction_3.append(P3_L)

                if Mode == "STEAM" or Mode == "All":

                    MC_1 = []
                    MC_2 = []
                    MC_3 = []

                    num_samples = 1000
                    ## Range
                    random_noise_range = [0, 0.004/2, 2]   ## Mean, sigma, ppm,  4mm + 2ppm (2 sigma)  ISO17123-3
                    ## Angles
                    random_noise_angle = [0, 0.000024241/5*4/2]    # Mean, sigma, 5"=0.000024241 precision datasheet  (2 sigma)  ISO17123-3
                    ## Tilt compensator
                    random_noise_tilt = [0, 0.000002424/2]    # Mean, sigma, 0.5"=0.000002424 precision datasheet  (2 sigma)  ISO17123-3

                    ## Compute uncertainty
                    T1, D1, A1, E1 = predict_u.data_training_L_Raw_data(t1, d1, a1, e1, index_1)
                    T2, D2, A2, E2 = predict_u.data_training_L_Raw_data(t2, d2, a2, e2, index_2)
                    T3, D3, A3, E3 = predict_u.data_training_L_Raw_data(t3, d3, a3, e3, index_3)

                    for i1,j1,k1,l1 in zip(T1,D1,A1,E1):
                        mu_raw_data, _,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, j1, random_noise_range, k1, l1, random_noise_angle, random_noise_tilt)
                        MC_1.append([i1,Tf_1@mu_raw_data,cov_matrix_simulated])
                    for i2,j2,k2,l2 in zip(T2,D2,A2,E2):
                        mu_raw_data, _,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, j2, random_noise_range, k2, l2, random_noise_angle, random_noise_tilt)
                        MC_2.append([i2,Tf_2@mu_raw_data,cov_matrix_simulated])
                    for i3,j3,k3,l3 in zip(T3,D3,A3,E3):
                        mu_raw_data, _,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, j3, random_noise_range, k3, l3, random_noise_angle, random_noise_tilt)
                        MC_3.append([i3,Tf_3@mu_raw_data,cov_matrix_simulated])

                    ## STEAM
                    MC_1_interpolated = theodo_g.STEAM_interpolation_with_covariance(T1, T_prediction_sensor, MC_1)
                    MC_2_interpolated = theodo_g.STEAM_interpolation_with_covariance(T2, T_prediction_sensor, MC_2)
                    MC_3_interpolated = theodo_g.STEAM_interpolation_with_covariance(T3, T_prediction_sensor, MC_3)

                    if(len(MC_1_interpolated)>0 and len(MC_2_interpolated)>0 and len(MC_3_interpolated)>0):
                        Prediction_1.append(MC_1_interpolated)
                        Prediction_2.append(MC_2_interpolated)
                        Prediction_3.append(MC_3_interpolated)
                        T_prediction.append(T_prediction_sensor)

        print("Interpolation finished !")

/home/maxime/repos/RTS_Extrinsic_Calibration/contrib/../scripts/groundtruth_utils.py:419: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  min_1 = np.array(min_1)
/home/maxime/repos/RTS_Extrinsic_Calibration/contrib/../scripts/groundtruth_utils.py:420: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  min_2 = np.array(min_2)
/home/maxime/repos/RTS_Extrinsic_Calibration/contrib/../scripts/groundtruth_utils.py:421: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differen

Min median error [m]:  0.00029
RTS number  2  taken as main frame
[1, 2, 1, 1, 3, 2]
Number of data for theodolites: [4945 4681 4737]
Bad measures: 558
3836 3619 3690
47
0
1
2
3
4
5
6
7
Exception interpolation !
8
Exception interpolation !
9
Exception interpolation !
Exception extraction !
Exception interpolation !
10
Exception interpolation !
Exception interpolation !
Exception interpolation !
11
Exception extraction !
Exception extraction !
12
Exception extraction !
Exception interpolation !
13
14
Exception extraction !
15
Exception interpolation !
Exception interpolation !
16
Exception interpolation !
Exception interpolation !
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
Exception extraction !
45
46
Interpolation finished !


In [4]:
print(Prediction_1[0])

[[1645734878.1595333, array([10.83765914,  5.84143618,  0.54181818]), array([[ 1.79722933e-06,  1.97589365e-06, -2.52134955e-07],
       [ 1.97589365e-06,  2.20466095e-06, -2.78401890e-07],
       [-2.52134955e-07, -2.78401890e-07,  5.04613577e-08]])], [1645734878.3595333, array([10.83767106,  5.8414307 ,  0.54181865]), array([[ 9.26967120e-04,  6.49484153e-07, -5.18119212e-09],
       [ 6.49484153e-07,  9.28747082e-04, -1.11611396e-07],
       [-5.18119211e-09, -1.11611396e-07,  9.27550906e-04]])], [1645734878.5595334, array([10.83764307,  5.84141867,  0.54181471]), array([[ 1.70787794e-06,  1.80988925e-06, -2.38648658e-07],
       [ 1.80988925e-06,  2.04493099e-06, -2.61147100e-07],
       [-2.38648658e-07, -2.61147100e-07,  9.18062702e-08]])], [1645734878.7595334, array([10.83757968,  5.84140456,  0.54180805]), array([[ 7.16902527e-04,  1.00425882e-06, -9.70283760e-08],
       [ 1.00425882e-06,  7.17915100e-04, -1.57904570e-07],
       [-9.70283760e-08, -1.57904570e-07,  7.16598125e

In [13]:
fig = plt.figure()

ax = fig.add_subplot(1, 1, 1, projection='3d')

number = 0
for i in Prediction_1[1]:
    if(number<2):
        theodo_g.get_cov_ellipsoid(ax, i[1], i[2], nstd=2.0, alpha=0.1)
        ax.scatter(i[1][0], i[1][1], i[1][2], color="black", label="inter/extra-polated", s=2)
    number = number +1
theodo_g.set_axes_equal(ax)

<IPython.core.display.Javascript object>

In [15]:
print(Prediction_1[1][1])

[1645734921.7595334, array([19.94083857,  5.9965927 ,  0.58022408]), array([[ 2.20985451e-06,  1.93151086e-06, -1.63706685e-07],
       [ 1.93151086e-06,  1.77538628e-06, -1.44421744e-07],
       [-1.63706686e-07, -1.44421744e-07,  6.01211255e-08]])]
